In [3]:
# get ECUST 

import math
import os
import sys
import time

import glob
from xlrd import open_workbook
import numpy as np
import pandas as pd
import random
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.externals import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score

from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2
import itertools
import math
import os
from gabor_features import gabor_filter
from landmark_features import predictLandMarks


# calculate euclidian distance
def calculateDistance(pts,landmarkCords):
	x1,y1 = landmarkCords[pts[0]]
	x2,y2 = landmarkCords[pts[1]]
	distance = math.sqrt((x1-x2)**2 + (y1-y2)**2)
	return distance

# ratio between two distances
def calculateRatio(pts,landmarkCords):
	x1,y1 = landmarkCords[pts[0]]
	x2,y2 = landmarkCords[pts[1]]
	x3,y3 = landmarkCords[pts[2]]
	x4,y4 = landmarkCords[pts[3]]
	firstDistance = math.sqrt((x1-x2)**2 + (y1-y2)**2)
	secondDistance = math.sqrt((x3-x4)**2 + (y3-y4)**2)
	return firstDistance/secondDistance

def generateSymmetryFeatures(landmarkCords):
	attractivePoints = [0, 4, 8, 12, 16, 17, 21, 22, 26,27, 31, 35, 36, 39, 42, 45,  48, 51, 54, 57]
	combinations = itertools.combinations(attractivePoints, 4)
	ratios = []
	#comb_list = list(combinations)
	#print(comb_list[3264], comb_list[561], comb_list[2498], comb_list[528], comb_list[544]) # most attrracive 5 ratios
	for combination in combinations:
		ratios.append(calculateRatio(combination,landmarkCords))
	return ratios

def generateDistanceFeatures(landmarkCords):
	distancePoints = []
	for i in range(68):
		distancePoints.append(i)
	combinations = itertools.combinations(distancePoints, 2)
	distances = []
	#comb_list = list(combinations)
	#print (comb_list[1477],comb_list[1476],comb_list[2226],comb_list[1004],comb_list[307]) # most attractive 5 distances
	for combination in combinations:
		distances.append(calculateDistance(combination,landmarkCords))
	return distances


def cropFace(image):

	detector = dlib.get_frontal_face_detector()
	image = imutils.resize(image, width=500)
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

	# detect faces in the grayscale image
	rects = detector(gray, 1)

	(x, y, w, h) = face_utils.rect_to_bb(rects[0])
	crop_img = image[y:y+h,x:x+w]
	#cv2.imshow("s", crop_img)
	return crop_img

def scut5500_train_and_test_set():
    """
    SCUT-FBP5500 dataset creation
    """ 

    df = pd.read_csv("/home/ubuntu/train.txt", sep=" ", header=None)
    df.columns = ['image', 'score']
    df['gabor']=''
    df['ratios']=''
    df['distances']=''
    #for index, row in df.iterrows():
    for index, row in df.iterrows():
        img= row['image']
        print(img)
        if os.path.isfile(img): 
           print(img)
           image = cv2.imread(img)
           try:
               face = cropFace(image)
               gabor_features = gabor_filter(image)
               df.at[index,'gabor'] = str(gabor_features)
               landmarkCords = predictLandMarks(image)
               ratios = generateSymmetryFeatures(landmarkCords)
               distances = generateDistanceFeatures(landmarkCords)
               df.at[index,'ratios'] = str(ratios)
               df.at[index,'distances'] = str(distances)
           except:
                print('rty')
        else:
            print('no file')
            
         
        
    return df,df   
""""
    df2 = pd.read_csv("/home/ubuntu/test.txt", sep=" ", header=None)
    df2.columns = ['image', 'score']
    df2['features']=''
    df2['gabor']=''
    df2['ratios']=''
    df2['distances']=''
    dfy = df2.head(50)
    
    for index, row in dfy.iterrows():
        img= row['image']
        if os.path.isfile(img): 
           image = cv2.imread(img)
           print(row['image'])
           try: 
               face = cropFace(image)
               gabor_features = gabor_filter(image)
               dfy.at[index,'features'] = str(gabor_features)
               landmarkCords = predictLandMarks(image)
               ratios = generateSymmetryFeatures(landmarkCords)
               distances = generateDistanceFeatures(landmarkCords)
               dfy.at[index,'gabor'] = str(gabor_features)
               dfy.at[index,'ratios'] = str(ratios)
               dfy.at[index,'distances'] = str(distances)
           except:
                print('no face')
        else:
            print('no file')
"""         
    
train2, test2 = scut5500_train_and_test_set()
train2, test2
#train2 = train2[train2['features']!='']


train2.to_csv('train_ecust.csv')
#test2.to_csv('test_ecust.csv')

/home/ubuntu/mtcnn_aligned_images/female/midestern/jyotirmoy-gupta-fpBNEz7N14s-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/jyotirmoy-gupta-fpBNEz7N14s-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/raamin-ka-hdra4gfbznU-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/raamin-ka-hdra4gfbznU-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/ghaly-wedinly-K07jvi25Qxg-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/ghaly-wedinly-K07jvi25Qxg-unsplash.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/midestern/kunal-goswami-YHSohAq-PuI-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/kunal-goswami-YHSohAq-PuI-unsplash.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/midestern/model-5046435_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/model-5046435_1920.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/midestern/shahin-khalaji-Fwp2mRiD7N4-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images

/home/ubuntu/mtcnn_aligned_images/female/midestern/pexels-the-lazy-artist-gallery-1149840.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/pexels-the-lazy-artist-gallery-1149840.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/sandeep-swarnkar-o2Bl6Y9hPos-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/sandeep-swarnkar-o2Bl6Y9hPos-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/little-girl-5912824_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/little-girl-5912824_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/pexels-toni-ph-2040376.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/pexels-toni-ph-2040376.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/woman-1284347_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/woman-1284347_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/raamin-ka-4lQmQ_DBbNc-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/midestern/raamin-ka-4lQmQ_DBbNc-uns

/home/ubuntu/mtcnn_aligned_images/female/asian/girl-5934632_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/girl-5934632_1920.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/asian/pexels-meijii-2014864.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/pexels-meijii-2014864.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/ipad-1721428_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/ipad-1721428_1920.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/asian/girl-1718116_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/girl-1718116_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/pexels-tima-miroshnichenko-6614828.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/pexels-tima-miroshnichenko-6614828.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/asian-3752815_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/asian-3752815_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/23.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/23.jpg
/home/ubuntu

/home/ubuntu/mtcnn_aligned_images/female/asian/d9.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/d9.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/character-2454694_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/character-2454694_1920.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/asian/girl-1733349_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/girl-1733349_1920.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/asian/muslim-417317_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/muslim-417317_1920.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/asian/muhammad-ruqiyaddin-0BFQaFG9iR0-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/muhammad-ruqiyaddin-0BFQaFG9iR0-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/fashion-4257899_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/fashion-4257899_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/girl-1848949_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/girl-18489

/home/ubuntu/mtcnn_aligned_images/female/asian/le-minh-phuong-jGZITdFhmts-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/le-minh-phuong-jGZITdFhmts-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/isco-wcJOYUVOwDc-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/isco-wcJOYUVOwDc-unsplash.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/asian/mathias-huysmans-kWLwiYznS3M-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/mathias-huysmans-kWLwiYznS3M-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/agung-pratamah-Xdm08vqsFrs-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/agung-pratamah-Xdm08vqsFrs-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/aiony-haust-Nsze2HlxoZo-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/aiony-haust-Nsze2HlxoZo-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/pexels-tuấn-kiệt-jr-1391499.jpg
/home/ubuntu/mtcnn_aligned_images/female/asian/pexels-tuấn-kiệt-jr-13

...

/home/ubuntu/mtcnn_aligned_images/female/caucasian/anton-malanin-lxEqRDB9Ng8-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/anton-malanin-lxEqRDB9Ng8-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/girl-1773894_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/girl-1773894_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/model-4605248_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/model-4605248_1920.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/caucasian/woman-751236_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/woman-751236_1920.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/caucasian/model-2614569_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/model-2614569_1920.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/caucasian/woman-4134319_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/woman-4134319_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/beautiful-1844724_192

rty
/home/ubuntu/mtcnn_aligned_images/female/caucasian/pexels-ali-pazani-2787341.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/pexels-ali-pazani-2787341.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/toa-heftiba-U7NLqKoMmiQ-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/toa-heftiba-U7NLqKoMmiQ-unsplash.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/caucasian/d02.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/d02.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/girl-204327_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/girl-204327_1920.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/caucasian/bailey-mahon-jfPKQraxv0w-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/bailey-mahon-jfPKQraxv0w-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/francesca-zama-GVIesWuFdI8-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/francesca-zama-GVIesWuFdI8-unsplash.jpg
/home/ubuntu/mtcnn_aligned_i

rty
/home/ubuntu/mtcnn_aligned_images/female/caucasian/people-5060268_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/people-5060268_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/sebastian-pociecha-9AdKSLh2320-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/sebastian-pociecha-9AdKSLh2320-unsplash.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/caucasian/thoughtful-583373_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/thoughtful-583373_1920.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/caucasian/woman-1000538_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/woman-1000538_1920.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/caucasian/court-cook-TSZo17r3m0s-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/court-cook-TSZo17r3m0s-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/artem-ivanchencko-M9-VgKDrhDA-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/caucasian/artem-ivanchencko-M9-

rty
/home/ubuntu/mtcnn_aligned_images/female/indian/happy-4895044_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/happy-4895044_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/woman-5267153_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/woman-5267153_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/pexels-azraq-al-rezoan-5641188.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/pexels-azraq-al-rezoan-5641188.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/ursula-spaulding-_B_UiKIAo20-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/ursula-spaulding-_B_UiKIAo20-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/loren-joseph-11eY-tkB9VU-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/loren-joseph-11eY-tkB9VU-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/pexels-nandhu-kumar-2703082.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/pexels-nandhu-kumar-2703082.jpg
rty
/home/ubuntu/mtcnn_aligned_ima

/home/ubuntu/mtcnn_aligned_images/female/indian/beauty-4175607_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/beauty-4175607_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/pexels-mabel-lee-2739792.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/pexels-mabel-lee-2739792.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/indian/bride-5170727_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/bride-5170727_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/portrait-4773342_1920.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/portrait-4773342_1920.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/indian/juniar-baresi-1g-BQ3wkgWI-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/juniar-baresi-1g-BQ3wkgWI-unsplash.jpg
rty
/home/ubuntu/mtcnn_aligned_images/female/indian/zhang-kaiyv-voWjZ-NIrqo-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/zhang-kaiyv-voWjZ-NIrqo-unsplash.jpg
/home/ubuntu/mtcnn_aligned_images/female/indian/33.jpg
/

In [2]:
train2

,image,score,gabor,ratios,distances
0,/home/ubuntu/mtcnn_aligned_images/female/mides...,5.583333,"[16.249860491071427, 37.52845982142857, 23.233...","[1.0572631694913412, 0.6047907175879412, 0.550...","[23.08679276123039, 47.01063709417264, 71.1125..."
1,/home/ubuntu/mtcnn_aligned_images/female/mides...,7.700000,"[0.6401666135204082, 25.863659917091837, 1.160...","[0.9497822205531884, 0.538576852387902, 0.5200...","[22.02271554554524, 43.18564576337837, 63.6396..."
2,/home/ubuntu/mtcnn_aligned_images/female/mides...,7.380952,,,
3,/home/ubuntu/mtcnn_aligned_images/female/mides...,7.566667,,,
4,/home/ubuntu/mtcnn_aligned_images/female/mides...,7.523810,,,
5,/home/ubuntu/mtcnn_aligned_images/female/mides...,6.150000,"[1.725665656887755, 18.61423788265306, 3.96504...","[0.733217393340338, 0.42033934427407016, 0.441...","[16.1245154965971, 33.015148038438355, 49.1629..."
6,/home/ubuntu/mtcnn_aligned_images/female/mides...,7.833333,"[0.7033442283163265, 19.860909598214285, 0.949...","[1.3887413798426747, 0.7640726444183477, 0.535...","[24.73863375370596, 48.27007354458868, 72.5327..."
7,/home/ubuntu/mtcnn_aligned_images/female/mides...,6.285714,"[3.1364795918367347, 31.753467793367346, 6.168...","[1.0348713506452307, 0.5858857882109461, 0.570...","[23.021728866442675, 45.09988913511872, 66.370..."
8,/home/ubuntu/mtcnn_aligned_images/female/mides...,7.555556,,,
9,/home/ubuntu/mtcnn_aligned_images/female/mides...,8.000000,"[2.9147002551020407, 29.417111766581634, 6.358...","[1.4517893468724408, 0.7832590280312355, 0.599...","[26.1725046566048, 52.3450093132096, 77.103826..."


In [7]:
scut = pd.read_csv('train_scut.csv')
ecust = pd.read_csv('train_ecust.csv')

X_train =[]
y_train = []

for index, row in train2.iterrows():
    X_train.append(np.fromstring(row['gabor'][1:-1], dtype=np.float, sep=','))
    y_train.append(row['score'])

X_test =[]
y_test = []

for index, row in test2.iterrows():
    X_test.append(np.fromstring(row['gabor'][1:-1], dtype=np.float, sep=','))
    y_test.append(row['score'])


In [11]:
import pandas as pd
import numpy as np
scut = pd.read_csv('train_scut.csv')
ecust = pd.read_csv('train_ecust.csv')
ecust = ecust[ecust['gabor'].notnull()]   
#scut = scut[scut['ratios'].notnull()] 
#scut = scut[scut['distances'].notnull()]
ecust = ecust[ecust['gabor'].notnull()]   
#ecust = scut[ecust['ratios'].notnull()] 
#ecust = ecust[ecust['distances'].notnull()] 
ecust

,Unnamed: 0,image,score,gabor,ratios,distances
0,0,/home/ubuntu/mtcnn_aligned_images/female/mides...,5.583333,"[16.249860491071427, 37.52845982142857, 23.233...","[1.0572631694913412, 0.6047907175879412, 0.550...","[23.08679276123039, 47.01063709417264, 71.1125..."
1,1,/home/ubuntu/mtcnn_aligned_images/female/mides...,7.700000,"[0.6401666135204082, 25.863659917091837, 1.160...","[0.9497822205531884, 0.538576852387902, 0.5200...","[22.02271554554524, 43.18564576337837, 63.6396..."
5,5,/home/ubuntu/mtcnn_aligned_images/female/mides...,6.150000,"[1.725665656887755, 18.61423788265306, 3.96504...","[0.733217393340338, 0.42033934427407016, 0.441...","[16.1245154965971, 33.015148038438355, 49.1629..."
6,6,/home/ubuntu/mtcnn_aligned_images/female/mides...,7.833333,"[0.7033442283163265, 19.860909598214285, 0.949...","[1.3887413798426747, 0.7640726444183477, 0.535...","[24.73863375370596, 48.27007354458868, 72.5327..."
7,7,/home/ubuntu/mtcnn_aligned_images/female/mides...,6.285714,"[3.1364795918367347, 31.753467793367346, 6.168...","[1.0348713506452307, 0.5858857882109461, 0.570...","[23.021728866442675, 45.09988913511872, 66.370..."
...,...,...,...,...,...,...
1251,1251,/home/ubuntu/mtcnn_aligned_images/female/india...,6.809524,"[2.9979073660714284, 27.819913903061224, 5.298...",NaN,NaN
1253,1253,/home/ubuntu/mtcnn_aligned_images/female/india...,6.250000,"[1.3849848533163265, 21.145926339285715, 2.772...","[0.6201736729460423, 0.36901704423437265, 0.43...","[17.11724276862369, 35.0, 51.35172830587107, 6..."
1254,1254,/home/ubuntu/mtcnn_aligned_images/female/india...,6.666667,"[0.583366549744898, 29.0111009247449, 1.524135...","[1.2491763353601955, 0.6840802805365027, 0.567...","[24.186773244895647, 48.25971404805462, 71.700..."
1255,1255,/home/ubuntu/mtcnn_aligned_images/female/india...,7.000000,"[4.635861766581633, 31.855090082908163, 6.2455...","[0.6067257909132441, 0.37104798760427277, 0.43...","[17.11724276862369, 33.0, 49.49747468305833, 6..."


In [14]:

X_ecust =[]
y_ecust = []

for index, row in ecust.iterrows():
    X_ecust.append(np.fromstring(row['gabor'][1:-1], dtype=np.float, sep=','))
    y_ecust.append(row['score'])


In [12]:
scut.columns

Index(['Unnamed: 0', 'image', 'score', 'gabor', 'ratios', 'distances'], dtype='object')

In [16]:
from sklearn.svm import SVR
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_absolute_error as mae

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
X_train, X_test, y_train, y_test = train_test_split(X_ecust, y_ecust, test_size=0.5, random_state=42)
reg= SVR()
reg.fit(X_train,y_train)
result=reg.predict(X_test)
print(pearsonr(y_test,result))
print(np.corrcoef(y_test,result))

(0.35531430793212176, 1.127269710034401e-13)
[[1.         0.35531431]
 [0.35531431 1.        ]]


In [19]:
for i in range(len(y_test)):
    print(y_test[i], result[i])

7.416666666666668 6.459482790961131
6.222222222222222 6.769340078260819
5.285714285714286 6.641262779475278
4.55 7.012062947108454
7.183333333333334 7.187377523023242
6.933333333333334 7.126831030830666
4.61904761904762 6.688277598535706
8.142857142857142 7.07254645633744
7.733333333333332 7.063602307286845
7.45 6.777688168164094
7.0 6.807302523677826
7.925 7.1528425696919005
7.095238095238094 6.684957520028842
7.333333333333332 6.924066402852273
6.675 6.977393630258037
8.0 6.926377441204582
7.190476190476191 7.024344132443223
6.294642857142858 6.8807245598997415
6.904761904761906 6.898768253193504
7.761904761904763 6.968758188318718
6.36842105263158 6.703169566152523
7.8888888888888875 7.009771144228093
8.777777777777779 7.0086887931447635
7.904761904761906 6.782294423806116
4.571428571428571 7.006602351573734
8.222222222222221 6.709319192009573
7.0 6.470478412557056
7.166666666666668 6.48091617469729
7.666666666666668 7.049950368852077
7.083333333333332 6.673040403323472
7.3809523809

In [18]:
#Linear + PCA

from sklearn import linear_model
from sklearn import decomposition
regr = linear_model.LinearRegression()




regr.fit(X_train,y_train)
ratings_predict = regr.predict(X_test)
corr = np.corrcoef(ratings_predict, y_test)[0, 1]
corr

0.3945788475354163

In [ ]:
ratings_predict.sort()
ratings_predict